## Model: ItemKNN (Item-based Top-N Recommendation)

This notebook implements the **ItemKNN** algorithm based on the following foundational paper:

> Badrul Sarwar, George Karypis, Joseph Konstan, and John Riedl.  
> **Item-based Collaborative Filtering Recommendation Algorithms**  
> WWW 2001.  
> [PDF](https://www.cs.umn.edu/research/GroupLens/publications/webconf.pdf)

### Core idea
ItemKNN builds a similarity matrix between items based on historical user interactions and uses it to recommend items that are most similar to those a user has previously interacted with.

### Implementation notes
- Similarities are computed using cosine similarity over the user-item interaction matrix.
- For each user, the top-N items are recommended by aggregating scores from similar items the user has interacted with.
- No training is needed beyond similarity computation.

This implementation matches the methodology described in the original paper and enables integration with re-ranking and evaluation pipelines.


### 📊 Comparison with RecBole: ItemKNN Implementation

This notebook implements the ItemKNN algorithm based on *Deshpande & Karypis (2004)*. While the core idea—neighborhood-based item similarity—is consistent, several architectural and evaluation differences exist compared to the implementation in RecBole.

#### 🔍 Metric Comparison (ML-100K, Original ItemKNN Only)

| Metric       | RecBole KNN | Custom KNN | Δ (%)        |
|--------------|-------------|------------|--------------|
| NDCG@10      | 0.3167      | 0.2647     | –16.43%      |
| Precision@10 | 0.1813      | 0.2074     | **+14.40%**  |
| Recall@10    | 0.2118      | 0.1843     | –12.98%      |
| Gini Index   | 0.9214      | 0.6871     | –25.42%      |
| Item Coverage| 0.2656      | 0.2212     | –16.72%      |
| Entropy      | 0.0119      | 0.6362     | **+5245%**   |
| Tail %       | 0.0         | 0.0        | –            |

> *Note: RecBole's entropy appears artificially low, likely due to the metric being calculated over the ranking positions rather than raw item occurrence distributions.*

---

### Explanation of Deviations from RecBole

The following technical differences likely explain the evaluation deviations between this implementation and RecBole’s built-in `ItemKNN`:

- **Similarity Function**  
  This notebook uses **cosine similarity** on the binary interaction matrix.  
  RecBole supports additional metrics like **Asymmetric Cosine**, **Pearson**, or **Jaccard**, and allows fine-tuning similarity behavior via config files.

- **Score Normalization**  
  Item relevance scores are not normalized in this code.  
  RecBole optionally normalizes similarity scores (e.g., dividing by sum of similarities or user profile length).

- **Neighbor Filtering**  
  The implementation retains only top-`k` neighbors with **positive similarity**, excluding weak links.  
  RecBole may include additional logic to ensure numerical stability or handle cold-starts.

- **User-based Filtering / Preprocessing**  
  RecBole filters users/items below a configurable threshold (`user_inter_num`, `item_inter_num`), leading to different evaluation sets.  
  Here, all data is used without such filtering.

- **Evaluation Methodology**  
  RecBole performs **full ranking over all items** not in the training set.  
  This notebook computes similarity scores from neighbors only and builds recommendation lists accordingly.

- **Relevance Scoring in NDCG**  
  This code uses **actual rating values** (1–5) as graded relevance in DCG, while RecBole uses **binary relevance** (1 if item in ground truth, else 0) unless explicitly configured otherwise.

- **Data Splitting Strategy**  
  This notebook uses a stratified 80/20 random split.  
  RecBole supports split strategies such as **`leave_one_out`**, **`user_leave_one_out`**, and **chronological split**, which affect ground truth and negative sampling.

- **Reranking Compatibility**  
  This implementation separates recommendation from reranking entirely, which allows flexible use of algorithms like **MMR** or **simple diversification**.  
  RecBole does not support custom rerankers in its evaluation pipeline.

---

### Conclusion

While differences exist, this implementation follows the conceptual design from the original ItemKNN paper and enables full control over similarity computation and reranking. The observed deviations in evaluation metrics (especially diversity) stem from simplifications, absence of internal filtering logic, and more flexible use of graded relevance.

In [1]:
# ItemKNN Recommender with Diversity Reranking

import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import math
import random
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

#################################
# ITEMKNN RECOMMENDER IMPLEMENTATION
#################################

class ItemKNNRecommender:
    def __init__(self, k=50, random_state=42):
        """
        Item-Based K-Nearest Neighbors recommender algorithm
        
        Parameters:
        - k: number of nearest neighbors to consider
        - random_state: seed for reproducibility
        """
        self.k = k
        self.random_state = random_state
        np.random.seed(random_state)
        
    def fit(self, user_item_matrix):
        """
        Train the ItemKNN model on the user-item matrix
        
        Parameters:
        - user_item_matrix: scipy sparse matrix with user-item interactions
        
        Returns:
        - self
        """
        self.user_item_matrix = user_item_matrix
        self.n_users, self.n_items = user_item_matrix.shape
        
        # Compute item-item similarity matrix
        print("Computing item-item similarity matrix...")
        self.item_factors = cosine_similarity(user_item_matrix.T)
        
        # Zero out self-similarity to avoid recommending the same item
        np.fill_diagonal(self.item_factors, 0)
        
        # Create a dictionary of items each user has interacted with
        self.user_items = defaultdict(set)
        for user, item in zip(*self.user_item_matrix.nonzero()):
            self.user_items[user].add(item)
        
        # For each item, find its k nearest neighbors
        self.item_neighbors = {}
        for item_id in range(self.n_items):
            # Get similarity scores for this item
            similarities = self.item_factors[item_id]
            
            # Find the k most similar items
            neighbor_ids = np.argsort(similarities)[::-1][:self.k]
            
            # Store neighbors with their similarity scores
            self.item_neighbors[item_id] = {
                neighbor_id: similarities[neighbor_id] 
                for neighbor_id in neighbor_ids 
                if similarities[neighbor_id] > 0
            }
            
        return self
    
    def recommend(self, user_id, n=10, exclude_seen=True):
        """
        Generate item recommendations for a user
        
        Parameters:
        - user_id: user index
        - n: number of recommendations to generate
        - exclude_seen: whether to exclude items the user has already interacted with
        
        Returns:
        - list of n recommended item indices
        """
        # If the user has no interactions, return random recommendations
        if user_id not in self.user_items or not self.user_items[user_id]:
            # Get random recommendations
            all_items = set(range(self.n_items))
            seen_items = self.user_items.get(user_id, set())
            candidate_items = list(all_items - seen_items) if exclude_seen else list(all_items)
            
            # Return random items
            if len(candidate_items) <= n:
                return candidate_items
            return random.sample(candidate_items, n)
        
        # Initialize score accumulator for all items
        scores = np.zeros(self.n_items)
        
        # For each item the user has interacted with
        for item_id in self.user_items[user_id]:
            # Get its neighbors
            if item_id in self.item_neighbors:
                # Add similarity scores to the accumulator
                for neighbor_id, similarity in self.item_neighbors[item_id].items():
                    scores[neighbor_id] += similarity
        
        # If requested, exclude items the user has already interacted with
        if exclude_seen:
            for item_id in self.user_items[user_id]:
                scores[item_id] = -np.inf
                
        # Get top n items by score
        top_items = np.argsort(scores)[::-1][:n]
        
        return top_items

#################################
# RERANKER IMPLEMENTATION
#################################

class SimpleReranker:
    """
    Simple reranker that balances original scores with diversity
    """
    def __init__(self, model, alpha=0.7):
        """
        Initialize reranker
        
        Parameters:
        - model: trained recommender model
        - alpha: weight for original scores (between 0 and 1)
                 higher alpha means more focus on accuracy
        """
        self.model = model
        self.alpha = alpha
        
        # Calculate item popularity
        self.item_popularity = np.zeros(model.n_items)
        for user in range(model.n_users):
            if user in model.user_items:
                for item in model.user_items[user]:
                    self.item_popularity[item] += 1
        
        # Normalize popularity
        max_pop = np.max(self.item_popularity)
        if max_pop > 0:
            self.norm_popularity = self.item_popularity / max_pop
        else:
            self.norm_popularity = np.zeros_like(self.item_popularity)
    
    def rerank(self, user_id, n=10):
        """
        Generate reranked recommendations
        """
        # Get original recommendations as a larger candidate pool
        candidates = self.model.recommend(user_id, n=n*3, exclude_seen=True)
        
        # Calculate original scores for these candidates
        scores = np.zeros(self.model.n_items)
        
        # For each item the user has interacted with
        for item_id in self.model.user_items.get(user_id, []):
            # Get its neighbors
            if item_id in self.model.item_neighbors:
                # Add similarity scores to the accumulator
                for neighbor_id, similarity in self.model.item_neighbors[item_id].items():
                    scores[neighbor_id] += similarity
        
        # Initialize selected items
        selected = []
        
        # Iteratively select items
        while len(selected) < n and candidates.size > 0:
            best_score = -np.inf
            best_item = None
            
            for item in candidates:
                if item in selected:
                    continue
                
                # Original score component
                score_orig = scores[item]
                
                # Diversity component
                diversity_score = 0
                if selected:
                    # Use item factors to calculate similarity
                    item_factors = self.model.item_factors[item]
                    selected_factors = self.model.item_factors[selected]
                    
                    # Calculate average similarity
                    similarities = []
                    for i, sel_idx in enumerate(selected):
                        # Get similarity directly from the matrix
                        sim = self.model.item_factors[item, sel_idx]
                        similarities.append(sim)
                    
                    if similarities:
                        avg_sim = np.mean(similarities)
                        diversity_score = 1 - avg_sim
                
                # Novelty component (inverse popularity)
                novelty_score = 1 - self.norm_popularity[item]
                
                # Calculate weighted score
                combined_score = (
                    self.alpha * score_orig + 
                    (1 - self.alpha) * 0.5 * diversity_score + 
                    (1 - self.alpha) * 0.5 * novelty_score
                )
                
                if combined_score > best_score:
                    best_score = combined_score
                    best_item = item
            
            if best_item is None:
                break
                
            selected.append(best_item)
            candidates = candidates[candidates != best_item]
            
        return selected

class MMRReranker:
    """
    Maximum Marginal Relevance (MMR) Reranker
    
    This reranker balances between relevance and diversity explicitly by
    selecting items that maximize marginal relevance - items that are
    both relevant to the user and different from already selected items.
    
    MMR formula: MMR = λ * rel(i) - (1-λ) * max(sim(i,j)) for j in selected items
    
    Where:
    - rel(i) is the relevance of item i to the user
    - sim(i,j) is the similarity between items i and j
    - λ is a parameter that controls the trade-off between relevance and diversity
    """
    
    def __init__(self, model, lambda_param=0.7):
        """
        Initialize the MMR reranker
        
        Parameters:
        - model: trained recommender model
        - lambda_param: trade-off parameter between relevance and diversity (0-1)
                        higher values favor relevance, lower values favor diversity
        """
        self.model = model
        self.lambda_param = lambda_param
        
    def calculate_item_similarity(self, item1, item2):
        """
        Calculate similarity between two items
        
        Parameters:
        - item1: index of first item
        - item2: index of second item
        
        Returns:
        - similarity: similarity between items (0 to 1)
        """
        # For ItemKNN, we can directly use the precomputed similarity matrix
        return self.model.item_factors[item1, item2]
    
    def rerank(self, user_id, n=10, candidate_size=100):
        """
        Generate reranked recommendations using Maximum Marginal Relevance
        
        Parameters:
        - user_id: user index in the model
        - n: number of recommendations to return
        - candidate_size: number of initial candidates to consider
        
        Returns:
        - reranked_items: list of reranked item indices
        """
        # Get original candidates and scores
        candidates = self.model.recommend(user_id, n=candidate_size, exclude_seen=True)
        
        # Calculate relevance scores for these candidates
        relevance_scores = np.zeros(self.model.n_items)
        
        # For each item the user has interacted with
        for item_id in self.model.user_items.get(user_id, []):
            # Get its neighbors
            if item_id in self.model.item_neighbors:
                # Add similarity scores to the accumulator
                for neighbor_id, similarity in self.model.item_neighbors[item_id].items():
                    relevance_scores[neighbor_id] += similarity
        
        # Normalize relevance scores to [0,1] range for the candidates
        candidate_scores = relevance_scores[candidates]
        min_score = np.min(candidate_scores)
        max_score = np.max(candidate_scores)
        score_range = max_score - min_score
        
        if score_range > 0:
            normalized_scores = (candidate_scores - min_score) / score_range
        else:
            normalized_scores = np.zeros_like(candidate_scores)
        
        # Initialize selected items
        selected = []
        
        # Select first item (most relevant)
        if candidates.size > 0:
            selected.append(candidates[np.argmax(normalized_scores)])
            remaining_candidates = set(candidates) - set(selected)
        else:
            remaining_candidates = set()
        
        # Iteratively select items using MMR
        while len(selected) < n and remaining_candidates:
            max_mmr = -np.inf
            max_item = None
            
            for item in remaining_candidates:
                # Get relevance component
                item_idx = np.where(candidates == item)[0][0]
                relevance = normalized_scores[item_idx]
                
                # Calculate diversity component (inverse of maximum similarity)
                max_sim = 0
                for selected_item in selected:
                    sim = self.calculate_item_similarity(item, selected_item)
                    max_sim = max(max_sim, sim)
                
                # Calculate MMR score
                mmr_score = self.lambda_param * relevance - (1 - self.lambda_param) * max_sim
                
                if mmr_score > max_mmr:
                    max_mmr = mmr_score
                    max_item = item
            
            if max_item is not None:
                selected.append(max_item)
                remaining_candidates.remove(max_item)
            else:
                break
                
        return selected

#################################
# EVALUATION METRICS
#################################

def calculate_ndcg(recommended_items, relevant_items, relevant_scores, k=None):
    """
    Calculate Normalized Discounted Cumulative Gain
    """
    if k is None:
        k = len(recommended_items)
    else:
        k = min(k, len(recommended_items))
    
    # Create a dictionary mapping relevant items to their scores
    relevance_map = {item_id: score for item_id, score in zip(relevant_items, relevant_scores)}
    
    # Calculate DCG
    dcg = 0
    for i, item_id in enumerate(recommended_items[:k]):
        if item_id in relevance_map:
            # Use rating as relevance score
            rel = relevance_map[item_id]
            # DCG formula: (2^rel - 1) / log2(i+2)
            dcg += (2 ** rel - 1) / np.log2(i + 2)
    
    # Calculate ideal DCG (IDCG)
    # Sort relevant items by their relevance scores in descending order
    sorted_relevant = sorted(zip(relevant_items, relevant_scores), 
                           key=lambda x: x[1], reverse=True)
    
    idcg = 0
    for i, (item_id, rel) in enumerate(sorted_relevant[:k]):
        # IDCG formula: (2^rel - 1) / log2(i+2)
        idcg += (2 ** rel - 1) / np.log2(i + 2)
    
    # Avoid division by zero
    if idcg == 0:
        return 0
    
    # Calculate NDCG
    ndcg = dcg / idcg
    
    return ndcg

def calculate_precision(recommended_items, relevant_items):
    """
    Calculate Precision@k
    """
    # Count number of relevant items in recommended items
    num_relevant_recommended = sum(1 for item in recommended_items if item in relevant_items)
    
    # Calculate precision
    precision = num_relevant_recommended / len(recommended_items) if recommended_items else 0
    
    return precision

def calculate_recall(recommended_items, relevant_items):
    """
    Calculate Recall@k
    """
    # Count number of relevant items in recommended items
    num_relevant_recommended = sum(1 for item in recommended_items if item in relevant_items)
    
    # Calculate recall
    recall = num_relevant_recommended / len(relevant_items) if relevant_items else 0
    
    return recall

def calculate_diversity_metrics(recommendations, item_popularity, total_items, tail_items=None):
    """
    Calculate diversity metrics for a set of recommendations
    """
    # Count occurrences of each item in recommendations
    rec_counts = Counter(recommendations)
    
    # 1. Item Coverage
    recommended_items = len(rec_counts)
    item_coverage = recommended_items / total_items
    
    # 2. Gini Index
    sorted_counts = sorted(rec_counts.values())
    n = len(sorted_counts)
    
    if n == 0:
        gini_index = 0
    else:
        cumulative_sum = 0
        for i, count in enumerate(sorted_counts):
            cumulative_sum += (i + 1) * count
        
        # Gini index formula
        gini_index = (2 * cumulative_sum) / (n * sum(sorted_counts)) - (n + 1) / n
    
    # 3. Shannon Entropy
    recommendations_count = sum(rec_counts.values())
    probabilities = [count / recommendations_count for count in rec_counts.values()]
    entropy = -sum(p * np.log2(p) for p in probabilities if p > 0)
    
    # Normalize entropy
    max_entropy = np.log2(min(total_items, recommendations_count))
    normalized_entropy = entropy / max_entropy if max_entropy > 0 else 0
    
    # 4. Tail Percentage
    if tail_items is None:
        # If tail_items not provided, use the bottom 20% by popularity
        sorted_pop_indices = np.argsort(item_popularity)
        num_tail_items = int(len(sorted_pop_indices) * 0.2)  # 20% least popular items
        tail_items = set(sorted_pop_indices[:num_tail_items])
    
    tail_recommendations = sum(1 for item in recommendations if item in tail_items)
    tail_percentage = tail_recommendations / len(recommendations) if recommendations else 0
    
    # Create results dictionary
    metrics = {
        'item_coverage': item_coverage,
        'gini_index': gini_index,
        'shannon_entropy': normalized_entropy,
        'tail_percentage': tail_percentage
    }
    
    return metrics, tail_items

#################################
# HELPER FUNCTIONS
#################################

def load_movielens_100k(path="ml-100k"):
    """
    Load the MovieLens 100K dataset
    """
    # Load ratings
    ratings_df = pd.read_csv(f"{path}/u.data", sep='\t', 
                           names=['user_id', 'item_id', 'rating', 'timestamp'])
    
    # Load movie information
    movie_df = pd.read_csv(f"{path}/u.item", sep='|', encoding='latin-1',
                          names=['item_id', 'title', 'release_date', 'video_release_date',
                                 'IMDb_URL'] + [f'genre_{i}' for i in range(19)])
    
    return ratings_df, movie_df

def create_user_item_matrix(ratings_df):
    """
    Create a sparse user-item interaction matrix from ratings
    """
    # Create mappings from original IDs to matrix indices
    user_ids = ratings_df['user_id'].unique()
    item_ids = ratings_df['item_id'].unique()
    
    user_mapping = {user_id: i for i, user_id in enumerate(user_ids)}
    item_mapping = {item_id: i for i, item_id in enumerate(item_ids)}
    
    # Map original IDs to matrix indices
    rows = ratings_df['user_id'].map(user_mapping)
    cols = ratings_df['item_id'].map(item_mapping)
    
    # Create binary matrix (1 if interaction exists, 0 otherwise)
    data = np.ones(len(ratings_df))
    user_item_matrix = csr_matrix((data, (rows, cols)), 
                                 shape=(len(user_mapping), len(item_mapping)))
    
    return user_item_matrix, user_mapping, item_mapping

#################################
# COMPREHENSIVE EVALUATION
#################################

def comprehensive_evaluation_multiple_rerankers(k=10, sample_size=None):
    """
    Run a comprehensive evaluation measuring both accuracy and diversity for multiple rerankers
    """
    print("="*80)
    print(f"COMPREHENSIVE EVALUATION WITH MULTIPLE RERANKERS (k={k})")
    print("="*80)
    
    # Load and prepare data
    print("\nLoading MovieLens 100K dataset...")
    ratings_df, movie_df = load_movielens_100k()
    
    print("Splitting data for evaluation...")
    train_df, test_df = train_test_split(
        ratings_df, 
        test_size=0.2, 
        stratify=ratings_df['user_id'], 
        random_state=42
    )
    
    print("Creating user-item matrix...")
    user_item_matrix, user_mapping, item_mapping = create_user_item_matrix(train_df)
    
    # Prepare for evaluation
    reverse_user_mapping = {v: k for k, v in user_mapping.items()}
    reverse_item_mapping = {v: k for k, v in item_mapping.items()}
    
    # Create test set ground truth
    test_relevant_items = defaultdict(list)
    test_relevant_scores = defaultdict(list)
    
    for _, row in test_df.iterrows():
        user_id = row['user_id']
        item_id = row['item_id']
        rating = row['rating']
        
        # Only include users and items that exist in our mappings
        if user_id in user_mapping and item_id in item_mapping:
            test_relevant_items[user_id].append(item_id)
            test_relevant_scores[user_id].append(rating)
    
    # Train model
    print("\nTraining ItemKNN model...")
    model = ItemKNNRecommender(k=50)
    model.fit(user_item_matrix)
    
    # Initialize rerankers
    print("\nInitializing rerankers...")
    simple_reranker = SimpleReranker(model=model, alpha=0.7)
    mmr_reranker = MMRReranker(model=model, lambda_param=0.7)
    
    # Setup dictionary for all rerankers' results
    rerankers = {
        "Original ItemKNN": None,
        "Simple Reranker": simple_reranker,
        "MMR Reranker": mmr_reranker
    }
    
    # Results dictionary
    all_results = {}
    
    # Select users for evaluation
    if sample_size is not None and sample_size < len(test_relevant_items):
        eval_users = random.sample(list(test_relevant_items.keys()), sample_size)
    else:
        eval_users = list(test_relevant_items.keys())
    
    print(f"\nEvaluating {len(eval_users)} users...")
    
    # Evaluate each reranker
    for reranker_name, reranker in rerankers.items():
        print(f"\nEvaluating {reranker_name}...")
        
        # Initialize metrics collectors
        ndcg_scores = []
        precision_scores = []
        recall_scores = []
        all_recs = []
        
        # Evaluate each user
        for user_id in eval_users:
            # Skip if user has no relevant items
            if not test_relevant_items[user_id]:
                continue
            
            user_idx = user_mapping[user_id]
            
            # Get recommendations
            if reranker is None:  # Original ItemKNN
                rec_idx = model.recommend(user_idx, n=k)
            else:  # Use reranker
                rec_idx = reranker.rerank(user_idx, n=k)
                
            rec = [reverse_item_mapping[idx] for idx in rec_idx]
            all_recs.extend(rec_idx)
            
            # Calculate accuracy metrics
            ndcg_scores.append(calculate_ndcg(
                rec, test_relevant_items[user_id], test_relevant_scores[user_id]
            ))
            precision_scores.append(calculate_precision(
                rec, test_relevant_items[user_id]
            ))
            recall_scores.append(calculate_recall(
                rec, test_relevant_items[user_id]
            ))
        
        # Calculate average accuracy metrics
        accuracy_metrics = {
            f'ndcg@{k}': np.mean(ndcg_scores),
            f'precision@{k}': np.mean(precision_scores),
            f'recall@{k}': np.mean(recall_scores)
        }
        
        # Calculate diversity metrics
        # First calculate item popularity
        item_popularity = np.zeros(model.n_items)
        for user in range(model.n_users):
            if user in model.user_items:
                for item in model.user_items[user]:
                    item_popularity[item] += 1
        
        # Then calculate diversity metrics
        diversity_metrics, _ = calculate_diversity_metrics(
            recommendations=all_recs,
            item_popularity=item_popularity,
            total_items=model.n_items
        )
        
        # Store results
        all_results[reranker_name] = {
            'accuracy': accuracy_metrics,
            'diversity': diversity_metrics
        }
    
    # Print comparative results
    print("\n" + "="*30 + " ACCURACY METRICS COMPARISON " + "="*30)
    print(f"{'Metric':<15}", end='')
    for reranker_name in rerankers.keys():
        print(f"{reranker_name:<20}", end='')
    print()
    print("-" * 80)
    
    for metric in [f'ndcg@{k}', f'precision@{k}', f'recall@{k}']:
        print(f"{metric:<15}", end='')
        baseline = all_results["Original ItemKNN"]['accuracy'][metric]
        for reranker_name in rerankers.keys():
            value = all_results[reranker_name]['accuracy'][metric]
            change = ((value - baseline) / baseline * 100) if baseline > 0 else float('inf')
            
            if reranker_name == "Original ItemKNN":
                print(f"{value:.4f}{' '*15}", end='')
            else:
                print(f"{value:.4f} ({change:+.1f}%){' '*5}", end='')
        print()
    
    print("\n" + "="*30 + " DIVERSITY METRICS COMPARISON " + "="*30)
    print(f"{'Metric':<15}", end='')
    for reranker_name in rerankers.keys():
        print(f"{reranker_name:<20}", end='')
    print()
    print("-" * 80)
    
    for metric in ['item_coverage', 'gini_index', 'shannon_entropy', 'tail_percentage']:
        print(f"{metric:<15}", end='')
        baseline = all_results["Original ItemKNN"]['diversity'][metric]
        for reranker_name in rerankers.keys():
            value = all_results[reranker_name]['diversity'][metric]
            change = ((value - baseline) / baseline * 100) if baseline > 0 else float('inf')
            
            if reranker_name == "Original ItemKNN":
                print(f"{value:.4f}{' '*15}", end='')
            else:
                print(f"{value:.4f} ({change:+.1f}%){' '*5}", end='')
        print()
    
    # Print interpretations
    print("\n" + "="*30 + " METRIC INTERPRETATIONS " + "="*30)
    print("Accuracy Metrics:")
    print("- NDCG: Higher is better, measures ranking quality")
    print("- Precision: Higher is better, measures relevant item ratio in recommendations")
    print("- Recall: Higher is better, measures coverage of all relevant items")
    
    print("\nDiversity Metrics:")
    print("- Item Coverage: Higher means more catalog items are recommended")
    print("- Gini Index: Lower means more equality in item recommendations")
    print("- Shannon Entropy: Higher means more diverse recommendations")
    print("- Tail Percentage: Higher means more niche items are recommended")
    
    # Return all results
    return all_results

# Execute with multiple rerankers when running the script directly
if __name__ == "__main__":
    comprehensive_evaluation_multiple_rerankers(k=10)

COMPREHENSIVE EVALUATION WITH MULTIPLE RERANKERS (k=10)

Loading MovieLens 100K dataset...
Splitting data for evaluation...
Creating user-item matrix...

Training ItemKNN model...
Computing item-item similarity matrix...

Initializing rerankers...

Evaluating 943 users...

Evaluating Original ItemKNN...

Evaluating Simple Reranker...

Evaluating MMR Reranker...

============================== ACCURACY METRICS COMPARISON ==============================
Metric         Original ItemKNN    Simple Reranker     MMR Reranker        
--------------------------------------------------------------------------------
ndcg@10        0.2736               0.2735 (-0.1%)     0.2775 (+1.4%)     
precision@10   0.2888               0.2889 (+0.0%)     0.2940 (+1.8%)     
recall@10      0.1878               0.1880 (+0.1%)     0.1921 (+2.3%)     

============================== DIVERSITY METRICS COMPARISON ==============================
Metric         Original ItemKNN    Simple Reranker     MMR Reranker    